# MLflow 3 RC0

- [Docs](https://mlflow.org/docs/3.0.0rc0/mlflow-3/)

In [22]:
import mlflow
import openai

mlflow.openai.autolog()

client = openai.OpenAI()

mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("mlflow3-rc0")

with mlflow.start_run():
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "Hello, world!"}],
    )

🏃 View run bemused-squid-440 at: http://localhost:5001/#/experiments/108756872616911814/runs/1c26f8f9f948406fa3d02f06e4f9bd8f
🧪 View experiment at: http://localhost:5001/#/experiments/108756872616911814


Trace(request_id=f20db72cc32e4ac8ac4d750554818a46)

## Register a Prompt


In [23]:
prompt = mlflow.register_prompt(
    name="summarization-prompt",
    template="""Summarize the following text at the provided level of complexity, where 1 represents a non-technical summary 
for novices and 5 represents a technical summary for experts.\n\nLevel: {{level}}\n\nText: {{text}}""",
    # Optional: Provide a commit message to describe the changes
    commit_message="Initial commit",
    # Optional: Specify any additional metadata about the prompt version
    version_metadata={
        "author": "Daniel Liden",
    },
    # Optional: Set tags applies to the prompt (across versions)
    tags={
        "task": "summarization",
        "language": "en",
    },
)

2025/04/21 11:27:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for prompt version to finish creation. Prompt name: summarization-prompt, version 4


## Invoke the Model with the Registered Prompt

In [24]:
import requests
from markdownify import markdownify
# helper function to get text from a webpage

def webpage_to_markdown(url):
    # Get webpage content
    response = requests.get(url)
    html_content = response.text
    
    # Convert to markdown
    markdown_content = markdownify(html_content)

    return markdown_content

url = "https://mlflow.org/docs/3.0.0rc0/tracing/tracing-schema"
markdown_content = webpage_to_markdown(url)

# Invoke the model with the registered prompt
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt.format(level=1, text=markdown_content)}],
)

Trace(request_id=a0f04cdc82694283a0e91d732e40ef21)

In [25]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt.format(level=5, text=markdown_content)}],
)

Trace(request_id=4a6783c318b747808d6cdeacdb94be7d)

## Evaluate the Model

In [26]:
from mlflow.metrics.genai import faithfulness

faithfulness_metric = faithfulness(model="openai:/gpt-4o")

# Fetch the LoggedModel that's automatically created during autologging
logged_model = mlflow.last_logged_model()

with mlflow.start_run():
    for level in range(1, 6):   
        response = (
            client.chat.completions.create(
                messages=[
                {"role": "user", "content": prompt.format(level=level, text=markdown_content)}
            ],
            model="gpt-4o-mini",
            temperature=0.1,
            max_tokens=2000,
        )
        .choices[0]
        .message.content
    )

        faithfulness_score = faithfulness_metric(
            predictions=response, inputs = " ", context = markdown_content
        ).scores[0]

        # Log metrics and pass model_id to link the metrics
        mlflow.log_metrics(
            {
                "faithfulness": faithfulness_score,
            },
            model_id=logged_model.model_id,
        )

100%|██████████| 1/1 [00:02<00:00,  2.03s/it]

🏃 View run learned-whale-641 at: http://localhost:5001/#/experiments/108756872616911814/runs/6ed1f65db40b490cb66ad329b47e6dd0
🧪 View experiment at: http://localhost:5001/#/experiments/108756872616911814


[Trace(request_id=0a24be43341044f38eafcf89c727cad3), Trace(request_id=d1f4703d327543b3b1c84d6c8a4f84d0), Trace(request_id=70601957c79249708b017140661d8740), Trace(request_id=fd18fdda8e8f4476b4cdc1b1e68543f8), Trace(request_id=64e9d582d6e9462e9652ab24bbdc0470)]